# Course Project - NBA data analysis (Due: 20 Dec 2021)

Group BD

## Task-1
1. [40%] Your boss would like to know the top-3 players of the best [cost–performance ratio](https://en.wikipedia.org/wiki/Price%E2%80%93performance_ratio) in 2020/2021 season, where the cost and the performance can be calculated by
    - PERFORMANCE: [Efficiency equation](https://en.wikipedia.org/wiki/Efficiency_(basketball)#:~:text=The%20NBA%20publishes%20online%20all%20of%20the%20basic,FG%20%E2%88%92%20Missed%20FT%20-%20TO%29%20%2F%20GP) based on the statistics in the **(3) Kaggle 2020/2021 season data**.
        - You could replace this equation if there is any better option.
    - COST: Salary in talbe "Player_Salary" of **(1) Kaggle basketball dataset**.
    - You are welcome to develop any better eqaution(s) to estimate the CP ratio to rank the top-3 players.


In [1]:
import pandas as pd
import numpy as np
import sqlite3

"""
Efficiency equation：
(PTS + REB + AST + STL + BLK − Missed FG − Missed FT - TO) / GP
PTS: Points 得分
REB, OREB, DREB: (Rebound) 篮板, 进攻篮板, 防守篮板
AST： (Assist缩写) 助攻
STL: Steals 抢断、断球
BLK: Blocks 盖帽
FG: Field Goals 投球命中（次数）/投篮(总称)，包括两分、三分球的投篮
FT：Free Throws 罚球
TOV（或TO）: Turnovers 失误次数
GM, GP; GS: games played; games started
"""


con = sqlite3.connect('datasets/basketball.sqlite')
cur = con.cursor()

df = pd.read_csv('datasets/nba2021_per_game.csv')

cps = {}
rank = []


def Efficiency(r):
    return r['PTS'] + r['ORB'] + r['DRB'] + r['AST'] + r['STL'] + r['BLK'] - (r['FGA'] - r['FG']) \
           - (r['FTA'] - r['FT']) - r['TOV']


def get_salary(name):
    salarys = pd.read_sql_query("SELECT value FROM Player_Salary WHERE namePlayer = ?", con, params=(name,))
    return np.average(salarys) / 1000000 if len(salarys) != 0 else -1


for i in range(len(df)):
    record = df.iloc[i]
    EFF = Efficiency(record)
    salary = get_salary(record['Player'])
    if salary == -1:
        continue
    name = EFF / salary
    if record['Player'] not in cps.keys():
        cps[record['Player']] = []
    cps[record['Player']].append(name)

for name in cps.keys():
    if len(cps[name]) > 1:
        cps[name] = np.average(cps[name])
    rank.append([name, cps[name]])
list.sort(rank, key=lambda x: x[1], reverse=True)
print('the top-3 players are:')
for i in range(3):
    print(*rank[i])

the top-3 players are:
Chimezie Metu [94.0]
Mason Jones [81.26381484852426]
Yogi Ferrell [76.57795635957405]


## Task-2
2. [40%] Your boss would like to know the **winning secret** from the games in 2020/2021. Please give your finding by carefully anaylzing the data in the table "game" of **(1) Kaggle basketball dataset**.
    - For instance, you can say that the **winning secret** to win a game is to **score more than your rival** since the **correlation** between the **winning status (1)** and **score more than your rival (1)** is always ONE (strongly correlated). However, this is definitely a bad idea to tell this **fact** to your boss.
    - After a careful analysis, you may find that the [field goal percentage (FG%)](https://en.wikipedia.org/wiki/Field_goal_percentage) (or any other combinations) is the **winning secret**
    since bilibala bilibala...

In [2]:
import pandas as pd
import sqlite3

con = sqlite3.connect('datasets/basketball.sqlite')
cur = con.cursor()

# SQL select statement using sqlite3 function (returning a list)
cur.execute("SELECT * FROM game")
L = cur.fetchall()
# print("List length:", len(L))
# print(L[0])  # print the first record in the table "game"

# SQL select statement using Pandas
# print(str(pd.read_sql_query("SELECT * FROM game WHERE SEASON_ID=22020", con).columns.values).replace(' ', ', '))
df = pd.read_sql_query("SELECT * FROM game WHERE SEASON_ID=22020", con)
pd.set_option('display.max_columns', None)  # 设置输出列数不受限
for i in range(len(df)):  # 将胜负结果设为1或0
    if df.iloc[i, 7] == 'W':
        df.iloc[i, 7] = 1
    if df.iloc[i, 33] == 'W':
        df.iloc[i, 33] = 1
    if df.iloc[i, 7] == 'L':
        df.iloc[i, 7] = 0
    if df.iloc[i, 33] == 'L':
        df.iloc[i, 33] = 0
    if df.iloc[i, 7] is None:  # 无胜负结果的先置-1，不然下面转格式会报错
        df.iloc[i, 7] = -1
    if df.iloc[i, 33] is None:
        df.iloc[i, 33] = -1
df['WL_HOME'] = df['WL_HOME'].astype(int)
df['WL_AWAY'] = df['WL_AWAY'].astype(int)
for i in range(len(df)):  # 再把无结果的转回去
    if df.iloc[i, 7] == -1:
        df.iloc[i, 7] = None
    if df.iloc[i, 33] == -1:
        df.iloc[i, 33] = None

pearsonHome = df.corr()['WL_HOME'].sort_values(ascending=False)  # 默认使用pearson相关系数
homeRow = pearsonHome.index.tolist()
homeValue = pearsonHome.values.tolist()
pearsonAway = df.corr()['WL_AWAY'].sort_values(ascending=False)  # 默认使用pearson相关系数
awayRow = pearsonAway.index.tolist()
awayValue = pearsonAway.values.tolist()

homeStrong = []
homeMid = []
awayStrong = []
awayMid = []

for i in range(len(homeRow)):
    if 1 > homeValue[i] >= 0.5:
        homeStrong.append({homeRow[i]: round(homeValue[i], 3)})
    if 0.5 > homeValue[i] >= 0.3:
        homeMid.append({homeRow[i]: round(homeValue[i], 3)})
    if 1 > awayValue[i] >= 0.5:
        awayStrong.append({awayRow[i]: round(awayValue[i], 3)})
    if 0.5 > awayValue[i] >= 0.3:
        awayMid.append({awayRow[i]: round(awayValue[i], 3)})

print('For home team:')
print("\tStrong correlation between win:")
print('\t\t' + str(homeStrong).replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('\'', ''))
print("\tModerate correlation between win:")
print('\t\t' + str(homeMid).replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('\'', ''))
print('\nFor away team:')
print("\tStrong correlation between win:")
print('\t\t' + str(awayStrong).replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('\'', ''))
print("\tModerate correlation between win:")
print('\t\t' + str(awayMid).replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('\'', ''))


For home team:
	Strong correlation between win:
		PLUS_MINUS_HOME: 0.804, HOME_TEAM_WINS: 0.661, PTS_HOME_y: 0.52, PTS_HOME: 0.505
	Moderate correlation between win:
		FG_PCT_HOME: 0.463, FGM_HOME: 0.396, FG3_PCT_HOME: 0.396

For away team:
	Strong correlation between win:
		PLUS_MINUS_AWAY: 0.804, HOME_TEAM_LOSSES: 0.637
	Moderate correlation between win:
		FG_PCT_AWAY: 0.468, PTS_AWAY: 0.468, FGM_AWAY: 0.43, FG3_PCT_AWAY: 0.379
